In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision

import tqdm

device = torch.device("cuda" if torch.cuda.is_available else "cpu")

In [4]:
# hyperparams
in_channel = 3
num_classes = 10
learning_rate = 0.001
batch_size = 1024
num_epochs = 5

# define model

In [13]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
    
    def forward(self, x):
        return x

# load pretrained VGG and modify it
model = torchvision.models.vgg16(pretrained=False)

# replace model avgpool layer with identity
model.avgpool = Identity()
model.classifier = nn.Linear(512, 10)

model.to(device)

/home/matt/anaconda3/envs/pytorch2/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/matt/anaconda3/envs/pytorch2/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# load data

In [14]:
train_dataset = datasets.CIFAR10(root='dataset/', train=True, 
                                 transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


# train

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in tqdm.tqdm(range(num_epochs)):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # put data on device
        data = data.to(device)
        targets = targets.to(device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)
        losses.append(loss.item())

        # backward
        optimizer.zero_grad() # set all gradients to 0 for each batch
                              # to not save gradient for each minibatch
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f'Cost at epoch {epoch} is {sum(losses)/len(losses)}')

 20%|██        | 1/5 [00:04<00:18,  4.54s/it]

Cost at epoch 0 is 2.3347062188751844


 40%|████      | 2/5 [00:09<00:13,  4.53s/it]

Cost at epoch 1 is 2.3005486605118732


 60%|██████    | 3/5 [00:13<00:09,  4.56s/it]

Cost at epoch 2 is 2.306040029136502


 80%|████████  | 4/5 [00:18<00:04,  4.55s/it]

Cost at epoch 3 is 2.3025426329398644


100%|██████████| 5/5 [00:22<00:00,  4.55s/it]

Cost at epoch 4 is 2.302295071738107


# test

In [16]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_correct = 0
    num_samples = 0
    model.eval()

    # don't need to compute gradients when checking accuracy
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device)
            y = y.to(device)

            scores = model(x)
            _, preds = scores.max(1) # index of maximum value for second dim
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

check_accuracy(train_loader, model)

Checking accuracy on training data
Got 4999 / 50000 with accuracy 10.00
